# RAGAS (Retrieval Augmented Generation Assessment)> 用於 RAG 系統評估與合成資料生成的框架（精要摘要）

## 一、整體概述
RAGAS（Retrieval-Augmented Generation Assessment）是一個專門為 **RAG（檢索增強生成）系統**設計的評估與資料生成框架。它的核心目標是：  
在**不高度依賴人工標註**的情況下，系統性、可量化地評估 RAG 系統的品質，並自動生成高品質的測試資料集。

---

## 二、為何需要 RAGAS
傳統 LLM 評估方式難以準確衡量 RAG 系統，原因包括：
- 回答品質同時受 **檢索結果** 與 **生成模型** 影響
- 人工評估成本高、可重現性低
- 缺乏標準化指標來衡量「是否真的用對了檢索到的內容」

RAGAS 提供一套 **自動化、模型驅動（LLM-based）** 的解法。

---

## 三、核心評估指標（Metrics）

### 1. 檢索層（Retrieval）
- **Context Precision（上下文精準率）**  
  檢索到的內容中，有多少是真正與問題相關的
- **Context Recall（上下文召回率）**  
  與問題相關的必要資訊是否有被完整檢索到

### 2. 生成層（Generation）
- **Answer Relevance（答案相關性）**  
  回答是否真正回應了使用者的問題
- **Faithfulness（忠實度）**  
  回答內容是否完全基於提供的上下文，而非憑空編造
- **Answer Correctness（答案正確性）**  
  在有標準答案的情況下，評估回答是否正確

### 3. 關聯性評估方式
上述多數指標並非傳統 NLP 指標，而是透過 **LLM 作為評審者（LLM-as-a-Judge）** 來計算。

---

## 四、合成資料生成（Synthetic Data Generation）

### 1. 核心理念
RAGAS 可從原始文件中自動產生：
- 問題（Questions）
- 對應上下文（Contexts）
- 參考答案（Ground Truth）

用於測試與回歸評估。

### 2. 問題生成類型
- **Single-hop Questions**：單一文件或單一事實即可回答
- **Multi-hop Questions**：需結合多段資訊或跨文件推理
- **Reasoning-based Questions**：需要推論、比較或總結

### 3. 知識圖譜導向
透過將文件轉換為 **知識圖譜（Knowledge Graph）**，提升問題多樣性與覆蓋率。

---

## 五、系統架構與整合
RAGAS 設計為模組化架構，可輕易整合至：
- **LangChain**
- **LlamaIndex**
- 自建 RAG pipeline

支援：
- 批次評估
- CI / CD 中的自動回歸測試
- 不同模型與檢索策略的橫向比較

---

## Quick Start

In [ ]:
import os

os.chdir("../")

# 設定 API Key
from initialization import credential_init
credential_init()

In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.embeddings import HuggingFaceEmbeddings
from ragas import evaluate, experiment

In [ ]:
# Embedding model
embeddings = HuggingFaceEmbeddings("BAAI/bge-m3")

### OpenAI API Service

In [ ]:
# client = AsyncOpenAI()
# llm = llm_factory("gpt-4o-mini", client=client)

### Ollama Cloud Service

如何在RAGAS框架下使用Ollama雲端服務

In [ ]:
client = AsyncOpenAI(
    api_key=os.environ["OLLAMA_API_KEY"],
    base_url="https://ollama.com/v1"
)
ragas_llm = llm_factory("gpt-oss:120b-cloud", provider="openai", client=client)

### RAGAS after v0.4 API

LLM領域的API變化的很快，變得不快的框架大概都是快要死掉的。BUCKLE UP~~

Mock Data:

In [ ]:
from ragas import Dataset

dataset = Dataset(name="test_dataset", backend="local/csv", root_dir=".")
test_data = [
    {"user_input": "什麼是 RAG?", 
     "response": "RAG 是檢索增強生成。", 
     "retrieved_contexts": ["RAG 結合了檢索與生成技術..."], 
     "reference": "RAG 是結合檢索與生成的技術。"},
]

for sample in test_data:
    dataset.append(sample)
dataset.save()

建立實驗

In [ ]:
from pydantic import BaseModel
from ragas import experiment
from ragas.metrics.collections import Faithfulness, AnswerRelevancy

# Define experiment result structure
class ExperimentResult(BaseModel):
    faithfulness: float
    answer_relevancy: float

# Create experiment function
@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=ragas_llm)
    answer_relevancy = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)
    
    faith_result = await faithfulness.ascore(
        response=row.get("response"),
        retrieved_contexts=row.get("retrieved_contexts"),
        user_input=row.get("user_input")
    )

    relevancy_result = await answer_relevancy.ascore(
        user_input=row.get("user_input"),
        response=row.get("response")
    )

    return ExperimentResult(
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value
    )

# Run experiment
exp_results = await run_evaluation.arun(dataset)

In [ ]:
exp_results.to_pandas()

# Metrics

## 內容精準度 (Context Precision)內容精準度 
(Context Precision) 是一項衡量指標，用於評估檢索器（Retriever）針對給定查詢時，將檢索內容中相關區塊（Relevant Chunks）的排名優於無關區塊的能力。具體而言，它評估了檢索內容中相關區塊被置於排名頂部的程度。其計算方式為內容中每個區塊的 Precision@k 平均值。Precision@k 是指在排名第 $k$ 位時，相關區塊數量與排名第 $k$ 位總區塊數量的比例。

$$
\text{Context Precision@K} =
\frac{\sum_{k=1}^{K} \left(\text{Precision@k} \times v_k\right)}
{\text{Total number of relevant items in the top } K \text{ results}}
$$


$$
\text{Precision@k} =
\frac{\text{true positives@k}}
{\text{true positives@k} + \text{false positives@k}}
$$

The ContextPrecision metric evaluates whether retrieved contexts are useful for answering a question by comparing each context against a reference answer. Use this when you have a reference answer available.

In [ ]:
from openai import AsyncOpenAI
from ragas.llms import llm_factory
from ragas.metrics.collections import ContextPrecision

# Create metric
scorer = ContextPrecision(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=[
        "The Eiffel Tower is located in Paris.",
        "The Brandenburg Gate is located in Berlin."
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="Where is the Eiffel Tower located?",
    reference="The Eiffel Tower is located in Paris.",
    retrieved_contexts=[
        "The Brandenburg Gate is located in Berlin.",
        "The Eiffel Tower is located in Paris."
    ]
)
print(f"Context Precision Score: {result.value}")

In [ ]:
eval_results.to_pandas()

## 回答相關性 (Answer Relevance)
當一個回答能直接且妥善地解決原始問題時，即被視為具有相關性。重要的是，我們對回答相關性的評估並不考慮真實性 (Factuality)，而是針對回答內容不完整或包含冗餘細節的情況進行扣分。

你可以想像成回答是否對於問題有足夠的針對性，可以避免出現雞同鴨講，或是回答是專業的幹話。

為了計算此分數，系統會引導大型語言模型（LLM）針對生成的回答多次產生對應的問題，並測量這些生成問題與原始問題之間的平均餘弦相似度 (Mean Cosine Similarity)。其核心理念在於：如果生成的回答能準確解決初始問題，那麼 LLM 應該能夠從該回答中反向生成與原始問題一致的問題。

$$
\text{Answer Relevancy} =
\frac{1}{N}
\sum_{i=1}^{N}
\text{cosine similarity}(E_{g_i}, E_o)
$$

為了計算答案與給定問題之間的相關性，我們採用兩個步驟：

### 步驟一
使用大型語言模型（LLM）從生成的答案中逆向推導出 **n** 個問題變體。  
例如，針對第一個答案，LLM 可能會生成以下可能的問題：

- 問題一：法國位於歐洲的哪個區域？
- 問題二：法國在歐洲中的地理位置是什麼？
- 問題三：你能指出法國位於歐洲的哪個地區嗎？

### 步驟二
計算這些生成問題與實際問題之間的平均餘弦相似度。

### 核心概念
其核心概念在於：如果答案確實正確回應了問題，那麼僅根據該答案，就極有可能重建出原始問題。


In [ ]:
from ragas.metrics.collections import AnswerRelevancy

# Create metric
scorer = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)

# Evaluate
result = await scorer.ascore(
    user_input="When was the first super bowl?",
    response="The first superbowl was held on Jan 15, 1967"
)
print(f"Answer Relevancy Score: {result.value}")

## 忠實度（Faithfulness）

Faithfulness 指標用於衡量回應內容與所檢索上下文之間在事實上的一致性。其分數範圍為 0 到 1，分數越高代表一致性越好。

若回應中的所有主張都能由檢索到的上下文所支持，則該回應被視為是忠實的（faithful）。

計算方式如下：

1. 辨識回應中的所有主張。
2. 檢查每一項主張是否能從檢索到的上下文中推論得出。
3. 使用以下公式計算忠實度分數：

$$
\text{忠實度分數} = \frac{\text{回應中可由檢索上下文支持的主張數量}}{\text{回應中的主張總數}}
$$

In [ ]:
from ragas.metrics.collections import Faithfulness

# Create metric
scorer = Faithfulness(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="When was the first super bowl?",
    response="The first superbowl was held on Jan 15, 1967",
    retrieved_contexts=[
        "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
    ]
)
print(f"Faithfulness Score: {result.value}")

In [ ]:
result

Weird...

可以用faithfulness_check.py這個檔案用pycharm debug mode進行檢查。

得到下面的回答:

reason='The context states that the First AFL–NFL World Championship Game was played on January 15, 1967, but it does not use the term "Super Bowl"; therefore the statement cannot be directly inferred from the given information.'

缺乏關鍵字 "Super Bowl"。

In [ ]:
# Evaluate
result = await scorer.ascore(
    user_input="Where and when was Einstein born?",
    response="Einstein was born in Germany, Einstein was born on 1st March 1879.",
    retrieved_contexts=[
        "Albert Einstein (born 14 March 1879) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time"
    ]
)
print(f"Faithfulness Score: {result.value}")

## 噪音敏感度（Noise Sensitivity）

### 🔍 理解 RAGAS 中的噪音敏感度

噪音敏感度用於衡量生成模型（LLM）在面對檢索到的上下文中**不正確或不相關資訊**時，受到干擾的程度。然而，從原始碼的實作可以看出，這個指標是**具條件性的**，需要謹慎解讀。

---

### 1. 邏輯運算（懲罰計算）

根據 RAGAS 的實作方式，答案中的某一項主張，只有在**同時滿足三個布林條件**時，才會觸發「噪音懲罰（Noise Penalty）」。

對於一個主張 $c$，其懲罰可表示為：

$$
\text{Penalty}(c) = (\text{Context} \cap \text{GT}) \wedge (\text{Context} \cap \text{Answer}) \wedge \neg \text{GT}
$$

其中：

- $(\text{Context} \cap \text{GT})$：上下文必須是**相關的**（包含正確答案／事實）。
- $(\text{Context} \cap \text{Answer})$：模型必須對該上下文保持**忠實**（答案確實是從上下文中抽取的）。
- $\neg \text{GT}$：該主張必須是**錯誤的**（不存在於 Ground Truth 中）。

---

### 2. 「零分」的細微差異（Zero Score Nuance）

分數為 **0** 並不一定代表系統表現完美。由於使用的是 $\wedge$（AND）邏輯，只要鏈條中的**任何一個條件不成立**，最終分數就會變成 0。

| 如果…… | 結果 | 原因 |
| :--- | :--- | :--- |
| **檢索品質不佳** | 分數 = 0 | 若上下文不包含 Ground Truth，「相關性」條件為 False |
| **模型產生幻覺** | 分數 = 0 | 若答案不在上下文中，「忠實性」條件為 False |
| **模型表現完美** | 分數 = 0 | 若答案符合 Ground Truth，「錯誤性」條件為 False |

> **⚠️ 注意事項：**  
> 請務必將噪音敏感度與 **Context Recall（上下文召回率）** 及 **Faithfulness（忠實度）** 一併評估，以避免將「0 分」誤判為正向結果。


In [ ]:
from ragas.metrics.collections import NoiseSensitivity

# Create metric
scorer = NoiseSensitivity(llm=ragas_llm)

In [ ]:
result = await scorer.ascore(
    user_input="什麼是牛頓第二運動定律？它在物理上代表什麼意義？",
    
    response=(
        "牛頓第二運動定律指出，物體所受到的力會影響其運動狀態，"
        "力越大，物體運動變化越明顯。這個定律常用來解釋物體的運動行為。"
    ),
    
    reference=(
        "牛頓第二運動定律指出，作用在物體上的合力等於物體質量與加速度的乘積，"
        "可表示為 F = ma。此定律說明力是造成物體加速度的原因，"
        "並建立了力、質量與運動之間的定量關係，是經典力學的核心定律之一。"
    ),
    
    retrieved_contexts=[
        # 相關且正確的內容
        "牛頓第二運動定律說明，物體的加速度與所受合力成正比，與物體的質量成反比。",
        
        "在國中與高中物理中，常用 F = ma 來分析直線運動問題。",
        
        # 與主題相關但層次不同（輕微雜訊）
        "牛頓第一運動定律描述在無外力作用下，物體將維持靜止或等速直線運動。",
        
        # 明顯的雜訊內容
        "能量守恆定律指出，在封閉系統中，能量不會憑空產生或消失。",
    ]
)

In [ ]:
result

## 答案準確度（Answer Accuracy）

答案準確度用於衡量模型回應與給定問題之**參考標準答案（Ground Truth）**之間的一致程度。  
此指標透過兩個獨立的「**LLM-as-a-Judge**」提示進行評估，每個提示都會回傳一個評分（0、2 或 4）。

接著，該指標會將評分轉換為 \([0, 1]\) 的區間，並對兩位評審的分數取平均值。  
分數越高，表示模型的回答越接近參考答案。

評分對應意義如下：

- **0** → 回應不正確，或未回答與參考答案相同的問題。
- **2** → 回應與參考答案**部分一致**。
- **4** → 回應與參考答案**完全一致**。

In [ ]:
from ragas.metrics.collections import AnswerAccuracy
# Create metric
scorer = AnswerAccuracy(llm=ragas_llm)

# Evaluate
result = await scorer.ascore(
    user_input="牛頓第二運動定律的數學表示式是什麼？",
    
    response="牛頓第二運動定律的數學表示式為 F = ma。",
    
    reference="牛頓第二運動定律指出，物體所受合力等於質量乘以加速度，其數學表示式為 F = ma。"
)

print(f"Answer Accuracy Score: {result.value}")

In [ ]:
result = await scorer.ascore(
    user_input="光在真空中的速度約為多少？",
    response="光在真空中的速度約為 3 × 10^6 公尺每秒。",
    reference="光在真空中的速度約為 3 × 10^8 公尺每秒。"
)

print(f"Answer Accuracy Score: {result.value}")

# 測試集生成（Testset Generation）於 RAG

## 概述
我們將探討 Ragas 的測試集生成模組，用以為基於 **檢索增強生成（Retrieval-Augmented Generation, RAG）** 的問答機器人創建合成測試集。我們的目標是設計一個 **Ragas 航空助理**，能夠回答客戶在各種主題上的問題，包括： 

- 航班預訂 (Flight booking)
- 航班變更與取消  (Flight changes and cancellations_
- 行李政策  (Baggage policies)
- 查看預訂資訊 (Viewing reservations)
- 航班延誤 (Flight delays)
- 機上服務  (In-flight services)
- 特殊協助 (Special assistance)
 
為了確保我們的合成數據集盡可能真實且多樣化，我們將創建不同的客戶角色（customer personas）。每個角色將代表不同類型的旅客及其行為，有助於我們構建一個全面且具代表性的測試集。這種方法確保我們能夠徹底評估 RAG 模型的效果與穩健性。

**來源:** [Ragas 官方文件](https://docs.ragas.io/en/v0.4.0/howtos/applications/singlehop_testset_gen/)

In [ ]:
!git clone https://huggingface.co/datasets/vibrantlabsai/ragas-airline-dataset

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

path = "ragas-airline-dataset"
loader = DirectoryLoader(path, glob="**/*.md")
docs = loader.load()

In [ ]:
import os

os.chdir("../")

In [ ]:
from ragas.embeddings import HuggingFaceEmbeddings

from src.ollama_connection import ragas_ollama

ragas_llm = ragas_ollama(model='gpt-oss:120b-cloud')

embeddings = HuggingFaceEmbeddings("BAAI/bge-m3")

## 知識圖譜創建（Knowledge Graph Creation）

在我們希望從給定文件集中生成不同類型的查詢時，主要挑戰在於 **如何識別合適的文件或文檔區塊**，以便讓大型語言模型（LLMs）能夠生成查詢。為了解決這個問題，Ragas 採用了 **基於知識圖譜的測試集生成方法**。


![caption](knowledge_graph.jpg)

### 流程

1. 原始資料輸入 (Documents & Attributes)
- **文件集 (Documents)：** 系統的起點，包含大量非結構化的原始文本資料。  
- **內容屬性 (Content Attribute)：** 此階段的主要特徵是文件的純文字內容，準備進入處理流水線。

---

2. 資訊擷取階段 (Extraction)
- **擷取器 (Extractors)：** 透過自然語言處理（NLP）技術，系統會運行兩種核心擷取動作：  
  1. **實體擷取 (Entity extractor)：** 識別文本中的特定對象（如人名、地名、組織或核心概念）。  
  2. **主題擷取 (Topics extractor)：** 識別文件討論的核心主旨或分類標籤。  

- **帶有提取資料的節點：** 處理後，每份文件演變成一個包含「內容、實體、主題」多維屬性的資訊節點。

---

3. 關聯構建階段 (Relationship Builder)
- **相似性函數 (Similarity Functions)：** 系統使用兩大標準來決定不同節點之間是否存在聯繫：  
  1. **主題相似度 (Topic similarity)：** 檢查兩個節點是否討論相似的領域。  
  2. **實體重疊度 (Entity overlap)：** 檢查不同文件是否提到相同的關鍵實體。  

- **關係建立器 (Relationship Builder)：** 根據上述算法計算權重，若相似度達標，則在節點間建立連線（Edge）。

---

4. 最終知識圖譜生成 (Knowledge Graph)
- **圖譜結構：** 最終形成一個由節點（Node 1, Node 2...）與邊（Edges）組成的網路。  
- **標註連線：** 圖中的連線會標註數字 `(1)` 或 `(2)`，代表該關聯是基於「主題相似」還是「實體重疊」所建立的。  
- **層次與聚類：** 透過這種方式，原本孤立的文件被串聯成一個具備語義關聯的知識網路，便於後續的檢索與增強生成（RAG）。

In [ ]:
docs[0].metadata

快速建立一個Knowledge Graph.其中的Node只包含了原始數據，沒有其他的特徵和關聯

In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType


kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

kg

## 設定轉換流程 (Setup the Transforms)

### 擷取器 (Extractors)

不同的擷取器用於從每個節點中提取可用於建立節點間關聯的信息。例如，針對財務文件，可以使用的擷取器包括：

- **實體擷取器 (Entity Extractor)：** 提取實體，如公司名稱。  
- **關鍵詞擷取器 (Keyphrase Extractor)：** 提取每個節點中重要的關鍵詞或短語。  

你也可以撰寫自訂擷取器，以提取對你的領域相關的重要資訊。

> 擷取器可以基於大型語言模型（LLM Based），繼承自 `LLMBasedExtractor`；或者基於規則（Rule Based），繼承自 `Extractor`。

---

參考文件: [Ragas Transforms](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.SummaryCosineSimilarityBuilder)

在本節中，我們將使用僅由節點構建的知識圖譜來創建 **單跳查詢資料集 (Single Hop Query dataset)**。為了增強圖譜並改善查詢生成，我們應用了三個關鍵轉換流程：

1. **標題擷取 (Headline Extraction)：**  
   使用語言模型從每份文件中提取清晰的段落標題（例如，從 `flight cancellations.md` 中提取 “Airline Initiated Cancellations”）。這些標題可隔離特定主題，為生成精準問題提供直接上下文。

2. **標題拆分 (Headline Splitting)：**  
   根據擷取的標題將文件拆分為可管理的小節。此步驟增加節點數量，並確保生成更細緻、上下文更明確的查詢。

3. **關鍵詞擷取 (Keyphrase Extraction)：**  
   識別核心主題的關鍵短語（例如，重要座位資訊），作為語義種子點，增強生成查詢的多樣性與相關性。


In [ ]:
from ragas.testset.transforms import apply_transforms, HeadlinesExtractor, HeadlineSplitter, KeyphrasesExtractor

headline_extractor = HeadlinesExtractor(llm=ragas_llm, max_num=20)
headline_splitter = HeadlineSplitter(max_tokens=500)
keyphrase_extractor = KeyphrasesExtractor(llm=ragas_llm)

使用前兩份文件作為範例，接下來使用extractor處理

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

In [ ]:
output = [await headline_extractor.extract(node) for node in sample_nodes]
output[0]

將抽取出來的特徵放入node中

In [ ]:
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]
sample_nodes[0].properties

In [ ]:
sample_nodes[1].properties

### 關聯建立器 (Relationship Builder)

擷取到的信息用於建立節點之間的關聯。例如，在財務文件的情境中，可以根據節點中出現的實體來建立節點之間的關係。  

你也可以撰寫自訂的關聯建立器（Custom Relationship Builder），根據對你的領域相關的信息來建立節點間的關聯。

#### Jaccard 相似度 (Jaccard Similarity)

**定義：**  
Jaccard 相似度是一種衡量兩個集合之間相似性的指標，其公式為：

\[
\text{Jaccard Similarity} = \frac{|A \cap B|}{|A \cup B|}
\]

其中：  
- \(A\) 與 \(B\) 是兩個集合  
- \(|A \cap B|\) 表示兩集合的交集元素數量  
- \(|A \cup B|\) 表示兩集合的聯集元素數量  

**解釋：**  
Jaccard 相似度的值介於 0 到 1 之間：  
- 0 表示兩個集合完全不相交（完全不同）  
- 1 表示兩個集合完全相同  

**範例：**  
假設有兩個節點的關鍵詞集合：  

```python
A = {"flight", "booking", "cancellation"}
B = {"flight", "delay", "cancellation"}
```
交集： {"flight", "cancellation"} → 2
聯集： {"flight", "booking", "cancellation", "delay"} → 4

則 Jaccard 相似度為：

$$
|A \cap B| = 2, \quad |A \cup B| = 4
$$

$$
\text{Jaccard Similarity} = \frac{|A \cap B|}{|A \cup B|} = \frac{2}{4} = 0.5
$$

這表示這兩個節點的內容有 50% 的相似性。

應用場景：

在知識圖譜中，用於衡量節點之間的內容重疊度或主題相似度

用於文本、標籤或關鍵詞的比較

In [ ]:
from ragas.testset.transforms.relationship_builders.traditional import JaccardSimilarityBuilder

kg = KnowledgeGraph(nodes=sample_nodes)

rel_builder = JaccardSimilarityBuilder(property_name="headlines", new_property_name="entity_jaccard_similarity")
relationships = await rel_builder.transform(kg)

#### CosineSimilarity

In [ ]:
from ragas.testset.transforms.extractors import EmbeddingExtractor

embedding_extractor = EmbeddingExtractor(
    embedding_model=embeddings,
    embed_property_name="page_content",     # text source
    property_name="embedding" # where vectors are stored
)

In [ ]:
from ragas.testset.transforms.relationship_builders.cosine import CosineSimilarityBuilder

cosine_relationship_builder = CosineSimilarityBuilder(
    property_name="embedding",     # node attribute containing embeddings
    threshold=0.75,                # cosine similarity threshold
)

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

output = [await embedding_extractor.extract(node) for node in sample_nodes]
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]

In [ ]:
kg = KnowledgeGraph(nodes=sample_nodes)

relationships = await cosine_relationship_builder.transform(kg)

#### SummaryCosineSimilarity

In [ ]:
from ragas.testset.transforms.extractors import SummaryExtractor

summary_extractor = SummaryExtractor(
    llm=ragas_llm,
    property_name="summary" # where vectors are stored
)

embedding_extractor = EmbeddingExtractor(
    embedding_model=embeddings,
    embed_property_name="summary",     # text source
    property_name="summary_embedding" # where vectors are stored
)

In [ ]:
from ragas.testset.transforms.relationship_builders.cosine import SummaryCosineSimilarityBuilder

summary_cosine_relationship_builder = SummaryCosineSimilarityBuilder(property_name='summary_embedding')

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

In [ ]:
output = [await summary_extractor.extract(node) for node in sample_nodes]
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]

output = [await embedding_extractor.extract(node) for node in sample_nodes]
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, sample_nodes)]

In [ ]:
kg = KnowledgeGraph(nodes=sample_nodes)

relationships = await summary_cosine_relationship_builder.transform(kg)

In [ ]:
relationships

### Transforms
All of the components used to build the knowledge graph can be combined into a single transform that can be applied to the knowledge graph to build the knowledge graph. Transforms is made of up of a list of components that are applied to the knowledge graph in a sequence. It can also handle parallel processing of the components. The apply_transforms method is used to apply the transforms to the knowledge graph.

為了更好的理解Transforms的作用，我們會從簡單的開始，然後慢慢地一路加新的features進去

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms = [
    headline_extractor,
    # headline_splitter,
    # keyphrase_extractor
]

apply_transforms(kg, transforms=transforms)

In [ ]:
kg

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms = [
    headline_extractor,
    keyphrase_extractor
]

apply_transforms(kg, transforms=transforms)

In [ ]:
kg.nodes

In [ ]:
kg.relationships

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms = [
    headline_extractor,
    headline_splitter,
    keyphrase_extractor
]

apply_transforms(kg, transforms=transforms)

In [ ]:
kg.nodes

In [ ]:
kg.relationships

將 headline_splitter調到 transforms最後面

In [ ]:
sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms = [
    headline_extractor,
    keyphrase_extractor,
    headline_splitter,
]

apply_transforms(kg, transforms=transforms)

In [ ]:
kg.nodes

To apply few of the components in parallel, you can wrap them in Parallel class.

順序為
1. 抽取 headline
2. 分割headlines產生子節點
3. 同時進行keyphrase和page_content consine similarity抽取
4. 將節點通過cosine similarity和Jaccard similarity連結在一起

In [ ]:
# from ragas.testset.transforms import Parallel

page_content_embedding_extractor = EmbeddingExtractor(
    embedding_model=embeddings,
    embed_property_name="page_content",     # text source
    property_name="embedding" # where vectors are stored
)

cosine_relationship_builder = CosineSimilarityBuilder(
    property_name="embedding",     # node attribute containing embeddings
    threshold=0.75,                # cosine similarity threshold
)

# threshold `太高`(標準很模糊)，會導致沒有Jaccard連結
jaccard_relationship_builder = JaccardSimilarityBuilder(property_name="keyphrases", new_property_name="entity_jaccard_similarity", threshold=0.2)

sample_nodes = [Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[0].page_content, "document_metadata": docs[0].metadata}
                ),
                Node(type=NodeType.DOCUMENT,
                     properties={"page_content": docs[1].page_content, "document_metadata": docs[1].metadata}
                )]

kg = KnowledgeGraph(nodes=sample_nodes)

transforms = [
    headline_extractor,
    headline_splitter,
    Parallel(
        keyphrase_extractor,
        page_content_embedding_extractor,
    ),
    Parallel(
             cosine_relationship_builder,
             jaccard_relationship_builder)
]

apply_transforms(kg, transforms)

In [ ]:
kg.nodes

In [ ]:
kg.relationships

跑通全部的文件

In [ ]:
kg = KnowledgeGraph()

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

apply_transforms(kg, transforms)

Save Transformed Knowledge Graph

In [ ]:
kg.save("week_4/knowledge_graph.json")

load Knoweldge Graph

In [ ]:
kg_load = KnowledgeGraph()

kg_load.load("week_4/knowledge_graph.json")

## 為查詢生成設定角色（Personas）

角色提供情境與觀點，確保生成的查詢自然、符合使用者特性且具多樣性。透過依不同使用者視角量身打造查詢，我們的測試集得以涵蓋廣泛的使用情境：

- **首次搭機旅客（First Time Flier）**：生成具備詳細、逐步指引的查詢，滿足需要清楚說明的新手需求。
- **常客旅客（Frequent Flier）**：產出精簡、以效率為導向的查詢，適合經驗豐富的旅客。
- **憤怒的商務艙旅客（Angry Business Class Flier）**：產生帶有批判性與急迫語氣的查詢，反映其高度期待與立即解決問題的需求。

In [ ]:
from ragas.testset.persona import Persona

persona_first_time_flier = Persona(
    name="First Time Flier",
    role_description="Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.",
)

persona_frequent_flier = Persona(
    name="Frequent Flier",
    role_description="Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.",
)

persona_angry_business_flier = Persona(
    name="Angry Business Class Flier",
    role_description="Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.",
)

personas = [persona_first_time_flier, persona_frequent_flier, persona_angry_business_flier]

## 關係（Relationship Link）在測試資料生成中的使用時機總結

以下整理「關係連結（Relationship）」在 RAGAS 測試資料生成流程中的實際使用情境，以及 `SingleHopSpecificQuerySynthesizer` 如何依據節點屬性（property fields）產生資料。

---

## 1️⃣ 關係連結何時會被用於測試資料生成

### ❌ 不會使用關係的情境
在以下情況中，**關係連結完全不會被使用**：

- 使用 `SingleHopSpecificQuerySynthesizer`
- 關係類型為：
  - `cosine_similarity`
  - 向量距離
  - ANN / embedding-based 關係
- 關係僅包含數值型屬性（如相似度分數）

原因是：
- 單跳（Single-hop）查詢僅針對「單一節點」
- 測試問題必須能由該節點的文字內容直接回答
- 關係不構成可被詢問的語意內容

---

### ✅ 會使用關係的情境
關係連結僅在 **多跳（Multi-hop）測試資料生成** 時才會被使用，例如：

- `MultiHopSpecificQuerySynthesizer`
- 其他圖結構導向的 synthesizer

在這類情境中：
- Synthesizer 會沿著關係進行節點遍歷
- 組合多個節點的內容
- 生成需要跨節點推理的查詢

#### ✔ 常見且有意義的關係類型
- `same_topic`
- `related_to`
- `causes`
- `contrasts_with`
- `depends_on`

> ⚠️ 純數值關係（如 cosine similarity）本身通常不具備可提問的語意，除非經過語意轉換或自訂 prompt。

---

## 2️⃣ SingleHopSpecificQuerySynthesizer 如何使用節點屬性產生資料

`SingleHopSpecificQuerySynthesizer` 是 **以節點屬性為核心（property-centric）** 的資料生成器。

### 🔄 生成流程概覽
1. 隨機或依策略選擇一個節點（Node）
2. 讀取該節點中指定的文字屬性（`property_name`）
3. 將該屬性內容嵌入至 LLM Prompt
4. 要求 LLM 產生一個可由該內容直接回答的查詢

---

### 📌 可用的屬性特性
`property_name` 必須符合以下條件：

- 為 **文字型資料（string / text）**
- 具備語意完整性
- 能自然對應使用者可能提出的問題

#### ✔ 常見可用屬性
- `content`
- `summary`
- `headlines`
- `keyphrases`
- `faq`
- `description`

#### ❌ 不適合的屬性
- `embedding`
- `vector`
- `cosine_similarity`
- 任何數值或結構化向量資料

---

### 🧠 設計原則（Best Practices）

- **屬性回答問題，關係回答推理**
- Single-hop → 使用節點文字屬性
- Multi-hop → 使用節點內容 + 語意關係
- Embedding 與 similarity 僅用於「選擇資料」，而非「生成問題」

---

## 總結（TL;DR）

- 🔹 關係連結僅在 **多跳測試資料生成** 中被使用
- 🔹 `SingleHopSpecificQuerySynthesizer` 完全忽略關係
- 🔹 單跳問題僅依賴節點的文字屬性
- 🔹 Embedding 與 cosine similarity 不適合作為 query 生成來源
- 🔹 若要讓關係產生價值，需轉為具語意的關係或使用 multi-hop synthesizer

In [ ]:
from ragas.testset.synthesizers.single_hop.specific import SingleHopSpecificQuerySynthesizer

query_distibution = [
    (
        SingleHopSpecificQuerySynthesizer(llm=ragas_llm, 
                                          property_name="headlines", 
                                          name="headlines_synthesizer"),
        0.5,
    ),
    (
        SingleHopSpecificQuerySynthesizer(llm=ragas_llm, 
                                          property_name="keyphrases", 
                                          name="keyphrases_synthesizer"),
        0.5,
    ),
]

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=ragas_llm,
    embedding_model=embeddings,
    knowledge_graph=kg,
    persona_list=personas,
)

In [ ]:
testset = generator.generate(testset_size=10, query_distribution=query_distibution)

In [ ]:
import hashlib

df = testset.to_pandas()

def make_id(row):
    raw = f"{row['user_input']}|{row.get('reference', '')}"
    return hashlib.sha256(raw.encode("utf-8")).hexdigest()[:16]

df["testcase_id"] = df.apply(make_id, axis=1)

**QueryLength**
Available options are: 
- LONG
- MEDIUM
- SHORT

**QueryStyle((
Available options are: 
- MISSPELLED
- PERFECT_GRAMMAR
- POOR_GRAMMAR
- WEB_SEARCH_LIKE

## Run Evaluation with the synthetic dataset

In [ ]:
from ragas import Dataset

dataset = Dataset(name="test_dataset_synthesize", backend="local/csv", root_dir=".")

test_data = df.to_dict(orient='records')

for sample in test_data:
    dataset.append(sample)
dataset.save()

In [ ]:
dataset

### 建立experiment

In [ ]:
from pydantic import BaseModel
from ragas import experiment
from ragas.metrics.collections import Faithfulness, AnswerRelevancy, ContextPrecision, NoiseSensitivity


# Define experiment result structure
class ExperimentResult(BaseModel):
    faithfulness: float
    answer_relevancy: float
    context_precision: float
    noise_sensitivity: float
    testcase_id: str


# Create experiment function
@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=ragas_llm)
    answer_relevancy = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)
    context_precision = ContextPrecision(llm=ragas_llm)
    noise_sensitivity = NoiseSensitivity(llm=ragas_llm)
    
    faith_result = await faithfulness.ascore(
        response=row.get("response"),
        retrieved_contexts=row.get("retrieved_contexts"),
        user_input=row.get("user_input")
    )

    relevancy_result = await answer_relevancy.ascore(
        user_input=row.get("user_input"),
        response=row.get("response")
    )

    context_precision_result = await context_precision.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts")
    )

    noise_sensitivity_result = await noise_sensitivity.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts"),
        response=row.get("response")
    )
    
    return ExperimentResult(
        testcase_id=row.get("testcase_id"),
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value,
        context_precision=context_precision_result.value,
        noise_sensitivity=noise_sensitivity_result.value
    )
# Run experiment
# exp_results = await run_evaluation.arun(dataset)

### 使用LlamaIndex 完成 Response

檢索策略: 使用第二周的 SentenceWindowNodeParser

Loading directory with LlamaIndex

In [ ]:
from glob import glob

from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(
    input_files=glob("week_4/ragas-airline-dataset/*")
)

llama_index_docs = reader.load_data()

建立Index

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser, MarkdownNodeParser
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

callback_manager = CallbackManager([LlamaDebugHandler()])

sentence_splitter = SentenceSplitter.from_defaults(chunk_size=128, chunk_overlap=0)

# create the sentence window node parser w/ default settings
# applying sentence_splitter in SentenceWindowNodeParser: https://github.com/run-llama/llama_index/issues/14029
sw_parser = SentenceWindowNodeParser.from_defaults(
    window_size=2,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
    sentence_splitter=sentence_splitter.split_text
)

sw_index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[
        sw_parser],
    callback_manager=callback_manager,
    embed_model=embed_model
)

# add documents to index
for doc in llama_index_docs:
    sw_index.insert(doc)

In [ ]:
len(storage_context.docstore.docs)

使用GPT-oss:120b-cloud 作為生成模型

In [ ]:
from src.ollama_connection import llama_index_ollama

from llama_index.llms.ollama import Ollama
from openai import AsyncOpenAI

llama_index_llm = llama_index_ollama(model="gpt-oss:120b-cloud", temperature=0)

In [ ]:
# response = await llm.acomplete("What is the capital of France?")

建立query engine

In [ ]:
sw_query_engine = sw_index.as_query_engine(
    similarity_top_k=3,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
    llm=llama_index_llm
)

測試能不能正常跑

In [ ]:
response = await sw_query_engine.aquery(df.iloc[0]['user_input'])

In [ ]:
response.response

In [ ]:
raw_text = response.response
clean_text = raw_text.replace("\u202f", " ")
print(clean_text)

In [ ]:
response.source_nodes[0].metadata['window']

批次生成結果

In [ ]:
import asyncio
from datetime import datetime

from ragas import Dataset

async def process_row(row):
    output = await sw_query_engine.aquery(row.user_input)
    return {
        "response": output.response,
        "reference": row.reference,
        "user_input": row.user_input,
        "retrieved_contexts": [
            node.metadata["window"] for node in output.source_nodes
        ],
        "testcase_id": row.testcase_id
    }

dataset = Dataset(name="week_4/synthetic_benchmark", backend="local/csv", root_dir=".")

BATCH_SIZE = 2  # tune this based on rate limits / memory

rows = list(df.itertuples(index=False))

time_start = datetime.now()

for i in range(0, len(rows), BATCH_SIZE):
    batch = rows[i:i + BATCH_SIZE]
    results = await asyncio.gather(
        *(process_row(row) for row in batch)
    )
    for item in results:
        dataset.append(item)

time_end = datetime.now()

print(time_end-time_start)

dataset.save()

In [ ]:
len(dataset)

進行量測實驗

In [ ]:
# Create experiment function
sem = asyncio.Semaphore(2)

async def limited(coro):
    async with sem:
        return await coro

@experiment(ExperimentResult)
async def run_evaluation(row):
    faithfulness = Faithfulness(llm=ragas_llm)
    answer_relevancy = AnswerRelevancy(llm=ragas_llm, embeddings=embeddings)
    context_precision = ContextPrecision(llm=ragas_llm)
    noise_sensitivity = NoiseSensitivity(llm=ragas_llm)
    
    faith_task = faithfulness.ascore(
        response=row.get("response"),
        retrieved_contexts=row.get("retrieved_contexts"),
        user_input=row.get("user_input"),
    )

    relevancy_task = answer_relevancy.ascore(
        user_input=row.get("user_input"),
        response=row.get("response"),
    )

    context_precision_task = context_precision.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts"),
    )

    noise_sensitivity_task = noise_sensitivity.ascore(
        user_input=row.get("user_input"),
        reference=row.get("reference"),
        retrieved_contexts=row.get("retrieved_contexts"),
        response=row.get("response"),
    )

    (
        faith_result,
        relevancy_result,
        context_precision_result,
        noise_sensitivity_result,
    ) = await asyncio.gather(
        limited(faith_task),
        limited(relevancy_task),
        limited(context_precision_task),
        limited(noise_sensitivity_task),
    )
    
    return ExperimentResult(
        testcase_id=row.get("testcase_id"),
        faithfulness=faith_result.value,
        answer_relevancy=relevancy_result.value,
        context_precision=context_precision_result.value,
        noise_sensitivity=noise_sensitivity_result.value
    )

In [ ]:
exp_results = await run_evaluation.arun(dataset, name="benchmark_evaluation")

In [ ]:
exp_results.to_pandas()

In [ ]:
exp_results.to_pandas().to_csv("week_4/benchmark_evaluation.csv", index=False)

In [ ]:
dataset[0]['testcase_id']

In [ ]:
dataset[0]['user_input']

In [ ]:
dataset[0]['response']

In [ ]:
dataset[0]['retrieved_contexts']

In [ ]:
len(dataset[0]['retrieved_contexts'])